### Named Tuples

The ``namedtuple`` function in ``collections`` allows us to create a tuple that also has names attached to each field (aka property). This can be handy to reference data in the tuple structure by name instead of just relying on position.

The ``namedtuple`` function is basically a class factory that creates a new type of class that uses a tuple as its underlying data storage (in fact, named tuples inherit from `tuple`), but layers in field names to each position and makes a property out of the field name.

The ``namedtuple`` function creates a **class**, and we then use that class to instantiate our instances of named tuples.

To use the ``namedtuple`` function we therefore need to select a class **name**, as well as indicate the **property** names, in the order in which they will be stored and accessed in the tuple.

Note that a ``namedtuple``, like the regular ``tuple`` is an **immutable** data structure. (In fact, named tuples inherit from tuples - we'll revisit this in our section on metaclasses)

If you find yourself writing code such as:

In [1]:
class Point3D:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z

Forget it! You seriously need to use named tuples! Not only can you shorten the amount of code you need to write, but you get some additional functionality for "free", such as `__repr__` and `__eq__` that you do not have to implement yourself!

#### Creating Named Tuples

We are going to create a ``Point`` named tuple that will contain an x-coordinate and a y-coordinate.

In [2]:
from collections import namedtuple

In [3]:
Point2D = namedtuple('Point2D', ('x', 'y'))

Note that we have two different uses of `Point2D` here. The label we are assigning the return value of the call to ``namedtuple`` and the **name** of the class generated by calling ``namedtuple``.

We could also have done the following:

In [4]:
Pt = namedtuple('Point2D', ('x', 'y'))

The ``namedtuple`` class name is `Point2D`, but the label we `Pt` simply points to that class, so we would then create instances of the `Point2D` class as follows:

In [5]:
pt1 = Pt(10, 20)

And we can see what `pt1` is:

In [6]:
pt1

Point2D(x=10, y=20)

As you can see we have an object of type `Point2D`, and it has two properties, `x` and `y` with respective values `10` and `20`.

The only weird thing here is that we are using `Pt` to generate our instances of the `Point2D` class.

That's why we usually always created `namedtuple` generated classes this way:

In [7]:
Point2D = namedtuple('Point2D', ('x', 'y'))

Then the following makes more sense:

In [8]:
pt1 = Point2D(10, 20)

In [9]:
pt1

Point2D(x=10, y=20)

This is not different than doing this:

In [10]:
Pt3 = Point3D  # class we defined earlier

In [11]:
pt3 = Pt3(10, 20, 30)

In [12]:
pt3

As you can see above, we used another label `Pt3` as a label that also references the `Point3D` class. It would be weird to do it this way here, and its weird for tuples as well. Of course, you may run into circumstances where you need to do this - just not as a general rule.

Note that all named tuples  are honest to goodness **classes**, just as if you had used a `class` definition such as with `Point3D`. 

The `namedtuple` function **generates** classes for us - it is a **class factory**.

In [13]:
type(Point3D)

type

In [14]:
type(Point2D)

type

However, `Point2D` is a subclass of `tuple`, while `Point3D` is not:

In [15]:
isinstance(pt1, tuple)

True

In [16]:
isinstance(pt3, tuple)

False

So, when we create an instance of a class, we are in fact calling the `__new__` method with our initial values. It's just a callable that has the **field names** we used to generate our named tuple class as its parameters. This means we can use keyword arguments when instantiating our named tuples!

In [17]:
pt4 = Point2D(y=20, x=10)

In [18]:
pt4

Point2D(x=10, y=20)

#### What did we get for free using a named tuple vs our own class?

First using a named tuple for our 2D point:

In [19]:
pt2d_1 = Point2D(10, 20)
pt2d_2 = Point2D(10, 20)

In [20]:
pt2d_1

Point2D(x=10, y=20)

In [21]:
pt2d_1 == pt2d_2

True

Now using our 3D class:

In [22]:
pt3d_1 = Point3D(10, 20, 30)
pt3d_2 = Point3D(10, 20, 30)

In [23]:
pt3d_1

Oh, we probably need to implement the `__repr__` method in our class

In [24]:
pt3d_1 == pt3d_2

False

And we would also need to implement the __eq__ method!

Let's do that:

In [25]:
class Point3D:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
    
    def __repr__(self):
        return f"Point3D(x={self.x}, y={self.y}, z={self.z})"
    
    def __eq__(self, other):
        if isinstance(other, Point3D):
            return self.x == other.x and self.y == other.y and self.z == other.z
        else:
            return False

In [26]:
pt3d_1 = Point3D(10, 20, 30)
pt3d_2 = Point3D(10, 20, 30)

In [27]:
pt3d_1

Point3D(x=10, y=20, z=30)

In [28]:
pt3d_1 == pt3d_2

True

How about finding the largest coordinate in the point?

That's easy for `Point2D` since it is a tuple, but not the case for `Point3D`:

In [29]:
max(pt2d_1)

20

In [30]:
max(pt3d_1)

TypeError: 'Point3D' object is not iterable

How about calculating the dot product of two points (considering them as vectors starting at the origin)?

The formula would be:
a.b = a.x * b.x + a.y + b.y + a.z * b.z

For the 3D point we would need to do the following:

In [31]:
def dot_product_3d(a, b):
    return a.x * b.x + a.y * b.y + a.z + b.z

In [32]:
dot_product_3d(pt3d_1, pt3d_2)

560

But for our 2D point, which, remember is a tuple as well, we can write a generic function that would work equally well with a 3D named tuple too:

In [33]:
def dot_product(a, b):
    return sum(e[0] * e[1] for e in  zip(a, b))

Here's a break down of how we implemented the dot product:

First we zip up the components of `a` and `b` to get an iterable of tuples containing the x-coordinates in the 1st element, and the y-coordinates in the second tuple. Our zip will contain as many elements as there are dimensions.

In [34]:
a = Point2D(1, 2)
b = Point2D(10, 20)
print(a)
print(b)
print(tuple(a))
print(tuple(b))
print(list(zip(a, b)))

Point2D(x=1, y=2)
Point2D(x=10, y=20)
(1, 2)
(10, 20)
[(1, 10), (2, 20)]


Note that if we had more dimensions this would work equally well.

Suppose we had 3 dimensions:

In [35]:
u = (1, 2, 3)
v = (10, 20, 30)
list(zip(u, v))

[(1, 10), (2, 20), (3, 30)]

Then we create a comprehension that multiplies the components together:

In [36]:
[e[0] * e[1] for e in zip(a, b)]

[10, 40]

Then we simply add those up:

In [37]:
sum([e[0] * e[1] for e in zip(a, b)])

50

In [38]:
dot_product(a, b)

50

And if we defined a 4D point named tuple:

In [39]:
Point4D = namedtuple('Point4D', ['i', 'j', 'k', 'l'])

In [40]:
pt4d_1 = (1, 1, 1, 10)
pt4d_2 = (2, 2, 2, 10)

In [41]:
dot_product(pt4d_1, pt4d_2)

106

As you can see we got the correct dot product. We could not have done this using our `Point3D` class!

#### Other Ways to Specify Field Names

There are a number of ways we can specify the field names for the named tuple:

* we can provide a sequence of strings containing each property name
* we can provide a single string with property names separated by whitespace or a comma

In [42]:
Circle = namedtuple('Circle', ['center_x', 'center_y', 'radius'])

In [43]:
circle_1 = Circle(0, 0, 10)
circle_2 = Circle(center_x=10, center_y=20, radius=100)

In [44]:
circle_1

Circle(center_x=0, center_y=0, radius=10)

In [45]:
circle_2

Circle(center_x=10, center_y=20, radius=100)

Or we can do it this way:

In [46]:
City = namedtuple('City', 'name country population')

In [47]:
new_york = City('New York', 'USA', 8_500_000)

In [48]:
new_york

City(name='New York', country='USA', population=8500000)

This would work equally well:

In [49]:
Stock = namedtuple('Stock', 'symbol, year, month, day, open, high, low, close')

In [50]:
djia = Stock('DJIA', 2018, 1, 25, 26_313, 26_458, 26_260, 26_393)

In [51]:
djia

Stock(symbol='DJIA', year=2018, month=1, day=25, open=26313, high=26458, low=26260, close=26393)

In fact, since whitespace can be used we can even use a multi-line string!

In [52]:
Stock = namedtuple('Stock', '''symbol
                               year month day
                               open high low close''')

In [53]:
djia = Stock('DJIA', 2018, 1, 25, 26_313, 26_458, 26_260, 26_393)

In [54]:
djia

Stock(symbol='DJIA', year=2018, month=1, day=25, open=26313, high=26458, low=26260, close=26393)

#### Accessing Items in a Named Tuple

The major advantage of named tuples are that, as the name suggests, we can access the properties (fields) of the tuple by name:

In [55]:
pt1

Point2D(x=10, y=20)

In [56]:
pt1.x

10

In [57]:
circle_1

Circle(center_x=0, center_y=0, radius=10)

In [58]:
circle_1.radius

10

Now named tuples *are* tuples, so elements can be accessed by index, unpacked, and iterated.

In [59]:
circle_1[2]

10

In [60]:
for item in djia:
    print(item)

DJIA
2018
1
25
26313
26458
26260
26393


We can also unpack named tuples just like ordinary tuples:

In [61]:
pt1

Point2D(x=10, y=20)

In [62]:
x, y = pt1

In [63]:
print(x, y)

10 20


We can also use extended unpacking:

In [64]:
djia

Stock(symbol='DJIA', year=2018, month=1, day=25, open=26313, high=26458, low=26260, close=26393)

In [65]:
symbol, *_, close = djia

In [66]:
print(symbol, close)

DJIA 26393


And remember that the `_` we use in the unpacking is just a regular variable:

In [67]:
print(_)

[2018, 1, 25, 26313, 26458, 26260]


The field names for these named tuples can be any valid variable name **except** that they cannot start with an underscore. 

For example the following would not be valid:

In [68]:
Person = namedtuple('Person', ['firstname', 'lastname', '_age', 'ssn'])

ValueError: Field names cannot start with an underscore: '_age'

We can also choose to let the `namedtuple` function replace invalid field names automatically for us, by using the keyword argument `rename`. When we set that argument to `True` (it is `False` by default) it will replace the invalid name using the position (index) of the field, preceded by an underscore:

In [69]:
Person = namedtuple('Person', ['firstname', 'lastname', '_age', 'ssn'], rename=True)

In [70]:
eric = Person('Eric', 'Idle', 42, 'unknown')

In [71]:
eric

Person(firstname='Eric', lastname='Idle', _2=42, ssn='unknown')

As you can see the invalid field name `_y` was replaced by `_1` since it was the second element (i.e. index of `1`)

#### Named Tuple Internals

We can easily find out the fields in a named tuple using the `_fields` property:

In [72]:
Point2D._fields

('x', 'y')

In [73]:
Stock._fields

('symbol', 'year', 'month', 'day', 'open', 'high', 'low', 'close')

There is also a property, `_source` that allows us to see exactly the class that was generated by calling `namedtuple` (remember that `namedtuple` is a class **factory**):

In [74]:
print(Point2D._source)

from builtins import property as _property, tuple as _tuple
from operator import itemgetter as _itemgetter
from collections import OrderedDict

class Point2D(tuple):
    'Point2D(x, y)'

    __slots__ = ()

    _fields = ('x', 'y')

    def __new__(_cls, x, y):
        'Create new instance of Point2D(x, y)'
        return _tuple.__new__(_cls, (x, y))

    @classmethod
    def _make(cls, iterable, new=tuple.__new__, len=len):
        'Make a new Point2D object from a sequence or iterable'
        result = new(cls, iterable)
        if len(result) != 2:
            raise TypeError('Expected 2 arguments, got %d' % len(result))
        return result

    def _replace(_self, **kwds):
        'Return a new Point2D object replacing specified fields with new values'
        result = _self._make(map(kwds.pop, ('x', 'y'), _self))
        if kwds:
            raise ValueError('Got unexpected field names: %r' % list(kwds))
        return result

    def __repr__(self):
        'Return a nicely for

And of course this will be slightly different for another named tuple generated class:

In [75]:
print(Person._source)

from builtins import property as _property, tuple as _tuple
from operator import itemgetter as _itemgetter
from collections import OrderedDict

class Person(tuple):
    'Person(firstname, lastname, _2, ssn)'

    __slots__ = ()

    _fields = ('firstname', 'lastname', '_2', 'ssn')

    def __new__(_cls, firstname, lastname, _2, ssn):
        'Create new instance of Person(firstname, lastname, _2, ssn)'
        return _tuple.__new__(_cls, (firstname, lastname, _2, ssn))

    @classmethod
    def _make(cls, iterable, new=tuple.__new__, len=len):
        'Make a new Person object from a sequence or iterable'
        result = new(cls, iterable)
        if len(result) != 4:
            raise TypeError('Expected 4 arguments, got %d' % len(result))
        return result

    def _replace(_self, **kwds):
        'Return a new Person object replacing specified fields with new values'
        result = _self._make(map(kwds.pop, ('firstname', 'lastname', '_2', 'ssn'), _self))
        if kwds:
    

#### Converting Named Tuples to Dictionaries

The `namedtuple` generated class also provides us an instance method, `_asdict()` that will create a dictionary from all the fields in the named tuple:

In [76]:
eric._asdict()

OrderedDict([('firstname', 'Eric'),
             ('lastname', 'Idle'),
             ('_2', 42),
             ('ssn', 'unknown')])

Technically, it is an `OrderedDict` which we will cover in later section. Basically an `OrderedDict` is a dictionary that, unlike the standard built-in `Dictionary` is **guaranteed** to preserve the order of the keys.

[**Note** that as of Python 3.6, regular dictionaries **do** preserve the order of the keys, but until just recently it was not **guaranteed** and was bascially an implementation detail.

**However, this has now changed!!** Guido van Rossum has now agreed that this is no longer an implementation detail, and starting in Python 3.7 dictionary order is guaranteed. Since it is actually already the case in Python 3.6, you can now safely assume this fact - as long as you are running your code under Python 3.6 or higher. Your code will break if you rely on dictionary order prior to 3.6, in that case, still use an `OrderedDict`]

#### Overhead of Named Tuples

At this point you may be wondering whether there's more overhead to using a named tuple vs a regular tuple.

There is, but it is tiny. The field names are stored in the **class**, not every instance of the named tuples.
This means that the overhead incurred by the field names for one instance of the named tuple vs 1000 instances is the same. Otherwise, the instances are tuples, so you can access contained objects using indexing, slicing and iteration just as if it were a plain tuple. No overhead there either. Looking up values by name do have some overhead of course, but no more than if you had created a custom class.